# Emotion Recognition with BERT
Setting Environment

In [ ]:
# install ktrain on Google Colab
!pip install keras-nlp --upgrade
!pip install tensorflow_datasets

In [2]:
import keras_nlp
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
import urllib.request

emotions = urllib.request.urlopen(

   'https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/emotions.txt').read().decode('utf8').split('\n')
print(emotions)

## 1. Import Data

In [ ]:

'''
Reference-
    'admiration': 0,
    'amusement': 1,
    'anger': 2,
    'annoyance': 3,
    'approval': 4,
    'caring': 5, 
    'confusion': 6,
    'curiosity': 7,
    'desire': 8,
    'disappointment': 9,
    'disapproval': 10,
    'disgust': 11,
    'embarrassment': 12,
    'excitement': 13,
    'fear': 14,
    'gratitude': 15,
    'grief': 16,
    'joy': 17,
    'love': 18,
    'nervousness': 19,
    'optimism': 20,
    'pride': 21,
    'realization': 22,
    'relief': 23,
    'remorse': 24, 
    'sadness': 25,
    'surprise': 26, 
    'neutral': 27
'''
  

def preprocess_dataset(split, batch_size=128):

 def one_hot_encode(x):
   vec = tf.stack([x[emotion] for emotion in emotions], 0)
   print(vec)
   return x['comment_text'], tf.cast(vec, tf.uint8)

 def encode(x):
   vec = tf.stack([x[emotion] for emotion in emotions], 0)
   print(vec)
   vec1 = tf.argmax(vec)
   print(vec1)
   return x['comment_text'], tf.cast(vec1, tf.uint8)
 


 ds = tfds.load('goemotions', split=split)
 ds = ds.map(encode, num_parallel_calls=tf.data.AUTOTUNE)
 #ds = ds.map(tf.argmax, num_parallel_calls=tf.data.AUTOTUNE)
 ds = ds.shuffle(buffer_size=batch_size * 10)
 ds = ds.batch(batch_size, drop_remainder=False)
 ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
 return ds 

ds_splits = ['train', 'test', 'validation']
datasets = {split: preprocess_dataset(split) for split in ds_splits}
#datasets['train'] = [encoding[x] for x in datasets['train'][emotions]]

## 2. Data preprocessing and Training

* BERT can handle a maximum length of 512

In [ ]:
import keras
#preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased",sequence_length=350,num_classes=28)
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased", num_classes=28, preprocessor=preprocessor)
#classifier.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),)
#classifier.compile(loss=keras.losses.CategoricalCrossentropy(),)

#classifier.fit(x=X_train, y=to_categorical(y_train, 5), batch_size=6, validation_data=(X_test, to_categorical(y_test,5)), epochs=5)
#classifier.summary()
classifier.fit(datasets['train'], validation_data=datasets['validation'], epochs=10)
message = ['I just lost my job.']
prediction = classifier.predict(message)
print(prediction)

message = ['What an amazing weather it is today.']
prediction = classifier.predict(message)
print(prediction)

message = ['I was kicked out of my home at night']
prediction = classifier.predict(message)
print(prediction)

message = ['I went for a 5Km walk.']
prediction = classifier.predict(message)
print(prediction)


## 2. Validation


Evaluate classifier

In [ ]:
classifier.evaluate(datasets['test'])

107/107 [==============================] - 83s 654ms/step - loss: 0.7639 - sparse_categorical_accuracy: 0.7916


[0.7638999819755554, 0.7916298508644104]

#### Testing with other inputs

In [ ]:
message = ['I just lost my job.']
prediction = classifier.predict(message)
print(prediction)

message = ['What an amazing weather it is today.']
prediction = classifier.predict(message)
print(prediction)

message = ['I was kicked out of my home at night']
prediction = classifier.predict(message)
print(prediction)

message = ['I went for a 5Km walk.']
prediction = classifier.predict(message)
print(prediction)

## 4. Saving the model


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
path = F"/content/gdrive/MyDrive/bert_model_emotion_kerasNLP"
classifier.save(path)

In [ ]:
from keras.models import load_model
# returns the saved model
model = load_model(path)


In [ ]:
message = ['I just lost my job.']
prediction = model.predict(message)
print(prediction)

message = ['What an amazing weather it is today.']
prediction = model.predict(message)
print(prediction)

message = ['I was kicked out of my home at night']
prediction = model.predict(message)
print(prediction)

message = ['I went for a 5Km walk.']
prediction = model.predict(message)
print(prediction)